# PDBe API Training

This interactive Python notebook will guide you through various ways of programmatically accessing Protein Data Bank in Europe (PDBe) data using REST API

The REST API is a programmatic way to obtain information from the PDB and EMDB. You can access details about:

* sample
* experiment
* models
* compounds
* cross-references
* publications
* quality
* assemblies
and more...
For more information, visit https://www.ebi.ac.uk/pdbe/pdbe-rest-api

# Notebook #6
This notebook is the second in the training material series, and focuses on getting information for multiple PDB entries using the REST search API of PDBe.

## 1) Making imports and setting variables
First, we import some packages that we will use, and set some variables.

Note: Full list of valid URLs is available from https://www.ebi.ac.uk/pdbe/api/doc/

In [1]:
import requests # used for getting data from a URL
from pprint import pprint # pretty print
import matplotlib.pyplot as plt # plotting results
import pandas as pd # used for turning results into mini databases

# make graphs show on the page
%matplotlib inline

# use plotly and cufflinks to make interactive plots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)
cf.go_offline()


# settings for PDBe API
base_url = "https://www.ebi.ac.uk/pdbe/" # the beginning of the URL for PDBe's API.

api_base = base_url + "api/"

search_url = base_url + 'search/pdb/select?' # the rest of the URL used for PDBe's search API. 

## 2) a function to get data from the search API
Let's start with defining a function that can be used to GET data from the PDBe search API.


In [3]:
def make_request(search_term, number_of_rows=10):
    """
    This function can make GET requests to
    the PDBe search API
    
    :param url: String,
    :param pdb_id: String
    :return: JSON
    """
    search_variables = '&wt=json&rows={}'.format(number_of_rows)
    url = search_url+search_term+search_variables
    print(url)
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print("[No data retrieved - %s] %s" % (response.status_code, response.text))
    
    return {}

## 3) formatting the search terms 
This will allow us to use human readable search terms and this function will make a URL that the search API can handle.

In [29]:
def format_search_terms(search_terms, filter_terms=None):
    # print('formatting search terms: %s' % search_terms)
    search_string = ''
    filter_string = ''
    search_list = []
    if isinstance(search_terms, dict):
        for key in search_terms:
            term = search_terms.get(key)
            if ' ' in term:
                if not '[' in term:
                    if not '"' in term:
                        term = '"{}"'.format(term)
                    elif not "'" in term:
                        term = "'{}'".format(term)
            search_list.append('{}:{}'.format(key, term))
        search_string = ' AND '.join(search_list)
    else:
        if '&' in search_terms:
            search_string = search_terms.replace('&', ' AND ')
        else:
            search_string = search_terms
    if filter_terms:
        filter_string = '&fl={}'.format(','.join(filter_terms))
    # print('formatted search terms: %s' % search_string)
    final_search_string = 'q={}{}'.format(search_string, filter_string)
    print(final_search_string)
    return final_search_string

## 4) Getting useful data out of the search

This function will run the search and will return a list of the results

In [21]:
def run_search(search_terms, filter_terms=None, number_of_rows=100):
    search_term = format_search_terms(search_terms, filter_terms)

    response = make_request(search_term, number_of_rows)
    results = response.get('response', {}).get('docs', [])
    print('Number of results for {}: {}'.format(','.join(search_terms.values()), len(results)))
    return results
    

## 5) running a search

Now we are ready to actually run a search against the PDB API for entries containing human Dihydrofolate reductase in the PDB. This will return a list of results - only 10 to start with.

A list of search terms is available at:
https://www.ebi.ac.uk/pdbe/api/doc/search.html

This will return details of human Dihydrofolate reductase's in the PDB

The search terms are defined as a dictionary (a hash in other programming lanuguages). 
e.g. 
{"molecule_name":"Dihydrofolate reductase"}
Here we are searching for molecules named Dihydrofolate reductase.
If we search for two terms i.e. molecule_name and organism_scientific_name then we will get molecules that match both search terms.

We will return the number of results for two searches.

The first one will hit the limit of 100. There are more than 100 Dihydrofolate reductase structures. 
We have to add the argument "number_of_rows" to a higher number, say 1000, to find all the examples. 

In [5]:
print('1st search')
search_terms = {"molecule_name":"Dihydrofolate reductase"} 

results = run_search(search_terms)

1st search
https://www.ebi.ac.uk/pdbe/search/pdb/select?q=molecule_name:"Dihydrofolate reductase"&wt=json&rows=100
Number of results for Dihydrofolate reductase: 100


In [6]:
results = run_search(search_terms, number_of_rows=1000)

https://www.ebi.ac.uk/pdbe/search/pdb/select?q=molecule_name:"Dihydrofolate reductase"&wt=json&rows=1000
Number of results for Dihydrofolate reductase: 365


We will add organism_name of Human to the query to limit the results to only return those that are structures of Human Dihydrofolate reductase.

In [7]:
print('2nd search')
search_terms = {"molecule_name":"Dihydrofolate reductase",
                "organism_name":"Human"
               } 
results = run_search(search_terms)

2nd search
https://www.ebi.ac.uk/pdbe/search/pdb/select?q=molecule_name:"Dihydrofolate reductase" AND organism_name:Human&wt=json&rows=100
Number of results for Dihydrofolate reductase,Human: 79


We will then look at the last result.
We will print the data we have for the first result.

This will be the first item of the list "results"
i.e. results[0]

We are using "pprint" (pretty print) rather than "print" to make the result easier to read.

In [8]:
pprint(results[0])

a': 100.967,
 'q_nigli_cell_c': 56.376,
 'q_nigli_cell_gamma': 100.967,
 'q_nigli_cell_symmetry': 'R32',
 'q_num_interacting_entity_id': [0],
 'q_number_of_bound_entities': 2,
 'q_number_of_bound_molecules': 2,
 'q_number_of_copies': 1,
 'q_number_of_models': 1,
 'q_number_of_polymer_entities': 1,
 'q_number_of_polymer_residues': 374,
 'q_number_of_polymers': 1,
 'q_number_of_protein_chains': 1,
 'q_organism_name': ['Homo sapiens',
                     'Man',
                     'Homo Sapiens (Human)',
                     'Human',
                     'Homo Sapiens',
                     'Homo sapiens',
                     'Homo',
                     'Homininae',
                     'Hominidae',
                     'Primates',
                     'Mammalia',
                     'Chordata',
                     'Metazoa',
                     'Eukaryota'],
 'q_organism_scientific_name': ['Homo sapiens'],
 'q_organism_synonyms': ['Man',
                         'Homo Sapiens (Hum

As you can see we get lots of data back about the individual molecule we have searched for and the PDB entries in which it is contained. 

We can get the PDB ID and experimental method for this first row as follows.

In [9]:
print(results[0].get('pdb_id'))
print(results[0].get('experimental_method'))

1s3w
['X-ray diffraction']


We can restrict the results to only the information we want using a filter so its easier to see the information we want.

In [49]:
print('3rd search')
range = "[2010 TO 2019]"
range = range.strip('\"')
print(range)
search_terms = {"release_year":range,
                "status":"REL"
               }
filter_terms = ['pdb_id', 'experimental_method', 'release_year', 'resolution', 'assembly_mol_wt', 'synchrotron_site', 'journal']
results = run_search(search_terms, filter_terms)
pprint(results)

3rd search
[2010 TO 2019]
q=release_year:[2010 TO 2019] AND status:REL&fl=pdb_id,experimental_method,release_year,resolution,assembly_mol_wt,synchrotron_site,journal
https://www.ebi.ac.uk/pdbe/search/pdb/select?q=release_year:[2010 TO 2019] AND status:REL&fl=pdb_id,experimental_method,release_year,resolution,assembly_mol_wt,synchrotron_site,journal&wt=json&rows=100
Number of results for [2010 TO 2019],REL: 100
[{'assembly_mol_wt': 2196.434,
  'experimental_method': ['Electron Microscopy'],
  'journal': 'Nat. Struct. Mol. Biol.',
  'pdb_id': '5o2r',
  'release_year': 2017,
  'resolution': 3.4},
 {'assembly_mol_wt': 28.444,
  'experimental_method': ['X-ray diffraction'],
  'journal': 'J. Med. Chem.',
  'pdb_id': '4na8',
  'release_year': 2014,
  'resolution': 2.3},
 {'assembly_mol_wt': 79.563,
  'experimental_method': ['X-ray diffraction'],
  'journal': 'J. Virol.',
  'pdb_id': '4a8s',
  'release_year': 2012,
  'resolution': 2.9,
  'synchrotron_site': ['ESRF']},
 {'assembly_mol_wt': 2196

## 6) Analysing and plotting the results


We are going to use a Python package called Pandas to help us sort and visualise the results


First we have to do a bit of housekeeping, some of the results are lists (a PDB entry can have more than one experimental method or organism for example) so we need to change them into strings so we can use them in a graph

In [50]:
def change_lists_to_strings(results):
    """
    input - list of results from search
    output - list of results with lists changed into strings
    """
    for row in results:
        for data in row:
            if type(row[data]) == list:
                # if there are any numbers in the list change them into strings
                row[data] = [str(a) for a in row[data]]
                # unique and sort the list and then change the list into a string
                row[data] = ','.join(sorted(list(set(row[data]))))
                
    return results

In [51]:
results = change_lists_to_strings(results)
pprint(results)

[{'assembly_mol_wt': 2196.434,
  'experimental_method': 'Electron Microscopy',
  'journal': 'Nat. Struct. Mol. Biol.',
  'pdb_id': '5o2r',
  'release_year': 2017,
  'resolution': 3.4},
 {'assembly_mol_wt': 28.444,
  'experimental_method': 'X-ray diffraction',
  'journal': 'J. Med. Chem.',
  'pdb_id': '4na8',
  'release_year': 2014,
  'resolution': 2.3},
 {'assembly_mol_wt': 79.563,
  'experimental_method': 'X-ray diffraction',
  'journal': 'J. Virol.',
  'pdb_id': '4a8s',
  'release_year': 2012,
  'resolution': 2.9,
  'synchrotron_site': 'ESRF'},
 {'assembly_mol_wt': 2196.434,
  'experimental_method': 'Electron Microscopy',
  'journal': 'Nat. Struct. Mol. Biol.',
  'pdb_id': '5o2r',
  'release_year': 2017,
  'resolution': 3.4},
 {'assembly_mol_wt': 113.636,
  'experimental_method': 'X-ray diffraction',
  'journal': 'Proteins',
  'pdb_id': '4fio',
  'release_year': 2013,
  'resolution': 1.37,
  'synchrotron_site': 'Australian Synchrotron'},
 {'assembly_mol_wt': 2217.997,
  'experimental

Notice that the only thing that changed is ['X-ray diffraction'] is now 'X-ray diffraction'

If we wanted to know the experimental methods used to determine structures of Human Dihydrofolate reductase we could loop through the results and count how many entries use each experimental method. 

We can use a Python package called Pandas to do this for us. 
It changes the results into a mini database - called a DataFrame. 


In [52]:
def pandas_dataset(list_of_results):
    results = change_lists_to_strings(list_of_results) # we have added our function to change lists to strings
    df = pd.DataFrame(list_of_results)

    return df

df = pandas_dataset(list_of_results=results)
print(df)
    

    assembly_mol_wt  experimental_method  \
0          2196.434  Electron Microscopy   
1            28.444    X-ray diffraction   
2            79.563    X-ray diffraction   
3          2196.434  Electron Microscopy   
4           113.636    X-ray diffraction   
5          2217.997    X-ray diffraction   
6          2196.434  Electron Microscopy   
7          2196.434  Electron Microscopy   
8            54.070    X-ray diffraction   
9          2217.997    X-ray diffraction   
10         2217.997    X-ray diffraction   
11         2281.701    X-ray diffraction   
12         2196.434  Electron Microscopy   
13           89.883    X-ray diffraction   
14         2217.997    X-ray diffraction   
15         2217.997    X-ray diffraction   
16         2196.434  Electron Microscopy   
17         2196.434  Electron Microscopy   
18           35.494    X-ray diffraction   
19         2196.434  Electron Microscopy   
20         2281.701    X-ray diffraction   
21         3234.942  Electron Mi

We can use the this to count how many PDB codes there are for each experimental method
This groups PDB IDs by experimental method and then counts the number of unique PDB IDs per method.

In [56]:
ds = df.groupby('release_year')['pdb_id'].nunique()
ds_exp = df.groupby(['release_year','experimental_method'])['pdb_id'].nunique()

#Group X-ray entries:
df_xray = df[df['experimental_method'] == 'X-ray diffraction']
xray_year_pdb = df_xray.groupby('release_year')['pdb_id'].nunique()

#Group EM entries
df_em = df[df['experimental_method'] == 'Electron Microscopy']
em_year_pdb = df_em.groupby('release_year')['pdb_id'].nunique()

#By average resolution:
#Xray entries:
df_xray["resolution"].mean()
xray_year = df_xray.groupby('release_year')
xray_year_res = xray_year["resolution"].mean()
#EM entries:
df_em["resolution"].mean()
em_year = df_em.groupby('release_year')
em_year_res = em_year["resolution"].mean()
print(xray_year_pdb)

release_year
2010    2
2011    1
2012    7
2013    8
2014    8
2015    5
2016    8
2017    6
2018    2
2019    5
Name: pdb_id, dtype: int64


We can find which experimental method has the greatest (max) or lowest (min) number of entries.

In [15]:
dt = ds.max()
print(dt)
dt = ds.min()
print(dt)

78
1


We can sort the results so its in decending order and then the first value is the experimental method with the highest number of results

In [16]:
ds.sort_values(ascending=False).index[0]

'X-ray diffraction'

Or sort ascending so the experimental method with the lowest number of results is given

In [17]:
ds.sort_values(ascending=True).index[0]

'Solution NMR'

Or we can then very easily plot these results as a bar chart

In [18]:
ds.iplot(kind='bar')

We will make this into two functions so we can resue them

In [19]:
def pandas_count(list_of_results, column_to_group_by):
    df = pandas_dataset(list_of_results)
    ds = df.groupby(column_to_group_by)['pdb_id'].nunique()
    return ds
    
def pandas_min_max(list_of_results, column_to_group_by, get_min=True):
    df = pandas_dataset(list_of_results)
    if get_min:
        ds = df.groupby(column_to_group_by)['pdb_id'].min()
    else:
        ds = df.groupby(column_to_group_by)['pdb_id'].max()
    return ds
    
def pandas_plot(list_of_results, column_to_group_by, graph_type='bar'):
    ds = pandas_count(list_of_results=list_of_results, column_to_group_by=column_to_group_by)
    ds.iplot(kind=graph_type)


One for counting the results

In [20]:
pandas_count(list_of_results=results, column_to_group_by='experimental_method')

experimental_method
Solution NMR          1
X-ray diffraction    78
Name: pdb_id, dtype: int64

One for getting min or max

In [21]:
print('updated search')
search_terms = {"molecule_name":"Dihydrofolate reductase",
                "organism_name":"Human"
               }
filter_terms = ['pdb_id', 'resolution']
new_results = run_search(search_terms, filter_terms)
pandas_min_max(list_of_results=new_results, column_to_group_by='resolution')

updated search
https://www.ebi.ac.uk/pdbe/search/pdb/select?q=molecule_name:"Dihydrofolate reductase" AND organism_name:Human&fl=pdb_id,resolution&wt=json&rows=100
Number of results for Dihydrofolate reductase,Human: 79


resolution
1.050    1kmv
1.090    1kms
1.201    4m6j
1.210    5hsr
1.230    3fs6
1.240    3ghw
1.270    2w3b
1.300    3ghc
1.350    3ntz
1.396    4m6k
1.400    2c2s
1.450    3f8y
1.460    5hqy
1.500    2c2t
1.530    3gi2
1.550    6dav
1.600    2w3m
1.610    4qhv
1.620    4qjc
1.650    5hpb
1.700    3eig
1.760    4kfj
1.800    1mvt
1.840    4kbn
1.850    6a7e
1.900    1mvs
1.920    3n0h
2.000    1drf
2.010    3f8z
2.050    4ddr
2.060    6a7c
2.100    1boz
2.200    1pd9
2.300    1dhf
2.411    6de4
2.500    1ohj
2.715    4kd7
Name: pdb_id, dtype: object

and one for plotting the results

In [22]:
pandas_plot(list_of_results=results, column_to_group_by='experimental_method')

Remember this only searched through the first 10 results.
To increase the number of entries we have to run the search again, this time setting number_of_rows to a number in the function run_search. 

In [23]:
search_terms = {"molecule_name":"Dihydrofolate reductase",
                "organism_name":"Human"
               } 
results = run_search(search_terms, number_of_rows=10000)


https://www.ebi.ac.uk/pdbe/search/pdb/select?q=molecule_name:"Dihydrofolate reductase" AND organism_name:Human&wt=json&rows=10000
Number of results for Dihydrofolate reductase,Human: 79


Then we can count the results using our pandas function above

In [24]:
pandas_count(list_of_results=results, column_to_group_by='experimental_method')

experimental_method
Solution NMR          1
X-ray diffraction    78
Name: pdb_id, dtype: int64

Changing the result so it groups by release year of the PDB entries. 

In [25]:
pandas_count(list_of_results=results, column_to_group_by='release_year')

release_year
1990     2
1992     1
1995     2
1998     6
2002     2
2003     5
2004     3
2005     3
2007     2
2009    13
2010     7
2011     7
2012     1
2013     8
2014     1
2015     2
2017    10
2018     2
2019     2
Name: pdb_id, dtype: int64

And then plot the number of entries released per year

In [26]:
pandas_plot(list_of_results=results, column_to_group_by='release_year')

We can make this into a line graph 

In [27]:
pandas_plot(list_of_results=results, column_to_group_by='release_year', graph_type='line')

Try changing the term you want to search for and see if you get interesting results.

## 7) searching for two terms at once

It would be interesting to see how many PDB entries were solved by each experimental method per year. 

we can use the tag "release_year" to get the year of release of each entry

We have to define a new function to group entries by two terms.

When we do the search we have to filter the results by the terms we want to plot otherwise it takes too long to run. 

In [28]:
search_terms = {"all_enzyme_names":"Lysozyme",
               } 
filter_results = ['beam_source_name','release_year', 'pdb_id']
results = run_search(search_terms, filter_results, number_of_rows=10000)

https://www.ebi.ac.uk/pdbe/search/pdb/select?q=all_enzyme_names:Lysozyme&fl=beam_source_name,release_year,pdb_id&wt=json&rows=10000
Number of results for Lysozyme: 1865


This will take a while as it will return lots of results. 
We can then define a function to group the results by two terms. 

In [29]:
def pandas_plot_multi_groupby(results, first_column_to_group_by, second_column_to_group_by, y_axis='pdb_id', graph_type='line'):
    
    df = pandas_dataset(results)
    new_df = df.groupby([first_column_to_group_by, second_column_to_group_by])
    ds = new_df.count().unstack().reset_index(first_column_to_group_by)
    ds.iplot(x=first_column_to_group_by, y=y_axis, kind=graph_type)
    
def pandas_plot_multi_groupby_min(results, first_column_to_group_by, second_column_to_group_by, graph_type='line', use_min=False, use_max=False):
    
    df = pandas_dataset(results)
    new_df = df.groupby([first_column_to_group_by])[second_column_to_group_by]
    ds = None
    if use_min:
        ds = new_df.min()
    elif use_max:
        ds = new_df.max()
    else:
        print('specify either use_min or use_max')
        return None
    ds.plot(x=first_column_to_group_by, y=second_column_to_group_by, kind=graph_type)

def pandas_box_plot(results, first_column_to_group_by, second_column_to_group_by):
    df = pandas_dataset(results)
    df.boxplot(column=second_column_to_group_by,by=first_column_to_group_by)

In [30]:
pandas_plot_multi_groupby(results, 'release_year', 'beam_source_name')

This shows us that rotating anodes were used as the major source of radiation until around 2004 when Synchrotron's overtook as the major source of radiation. 

Try editing the queries to plot interesting trends within the PDB

## Questions to answer

1) Electron Microscopy is going through a revolution. Is this leading to a growth in Electron Microscopy PDB entries?

2) New refinement programs have got better and there are more methods to validate the quality of strucures in the PDB.
Have structures got better over time?
We can use "overall_quality" to judge this
This could be plotted as a groupby or a box plot. 

3) Electron Microscopy resolution has been said to be improving. Is this true? 
hint - the search term and filter can be different.
pandas_plot_multi_groupby_min with use_min would be useful to plot this or maybe a box plot?

3) It has been said that all the simple structures have been done and that only complicated structures are left. One metric for "complicated" could be size of the assembly - assembly_mol_wt would be useful here. 